# Simple Deep Learning model for semantic segmentation 
## Aerial Semantic Segmentation Drone Dataset

<font size="4">This dataset is provided by Graz University of Technology ==> [Aerial Semantic Segmentation Drone Dataset](http://dronedataset.icg.tugraz.at). It is really useful to gain wider knowledge about **Computer Vision** and **Deep Learning** techniques. In this notebook several concepts will be explained:
    <font>
- <font size="4">Tensorflow basics-intermediate commands<font>
- <font size="4">Dealing with **large Datasets**<font>
- <font size="4">Image processing with OpenCV<font>
- <font size="4">Creating **Pipelines** using data generators to avoid memory overrunning<font>
- <font size="4">Training, validation and testing sets<font>
- <font size="4">Building your own **Convolutional Neural Network (CNN)**<font>
- <font size="4">Familiarization with simple callbacks:<font>
    - <font size="4">EarlyStopping<font>
- <font size="4">Performance analysis - metrics: <font>
    - <font size="4">Confusion matrix<font>
    - <font size="4">**MIoU**...<font>
- <font size="4">Predictions visualization<font>

<font size='4'>The aim of this notebook is to perform semantic segmentation on drone images like the ones shown below.<font><font>

In [ ]:
import os, cv2

path = '../input/semantic-drone-dataset/dataset/semantic_drone_dataset/original_images/'

files = []

for i in os.listdir(path)[:9]:
    files.append(os.path.join(path, i))

img=[cv2.resize(cv2.imread(files[i], 3), (1200, 800)) for i in range(len(files))]

import matplotlib.pyplot as plt

fig, ax = plt.subplots(3, 3, figsize=(20,15))
for i in range(3):
    for j in range(3):
        ax[i, j].imshow(img[3*i+j])
        ax[i, j].get_xaxis().set_visible(False)
        ax[i, j].get_yaxis().set_visible(False)
plt.show()

# INDEX


- [<font size="4">Setup<font>](#section-one)
- [<font size="4">Analysing the dataset<font>](#section-two)
- [<font size="4">Dataset Pipeline for preprocessing<font>](#section-three)
- [<font size="4">Convolutional Neural Network Model and Training<font>](#section-four)
- [<font size="4">Model performance<font>](#section-five)


<a id="section-one"></a>
# 1. Setup 

 <font size="4">As always, it is necessary to import **required libraries**, just basic setting up.<font>

In [ ]:
import numpy as np
import cv2
import os
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import Sequence
import random
import itertools
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

<a id="section-two"></a>
# 2. Analysing the dataset

<font size="4">Firstly, we should take a look to our dataset, analyse the folders structure as well as the content of each of the dataset files.
It can be seen that label images (or masks) store information from the label of each pixel as a number corresponding to each class from 0 to 23 and this is the reason why the images seems to be too dark (0-255).

In [ ]:
img = cv2.imread('../input/semantic-drone-dataset/dataset/semantic_drone_dataset/original_images/002.jpg', 3)
label = cv2.imread('../input/semantic-drone-dataset/dataset/semantic_drone_dataset/label_images_semantic/002.png', 3)
rgb = cv2.imread('../input/semantic-drone-dataset/RGB_color_image_masks/RGB_color_image_masks/002.png', 3)
fig, ax = plt.subplots(1, 3, figsize=(20,6))
ax[0].imshow(img)
ax[1].imshow(label)
ax[2].imshow(rgb)
ax[0].set_title('Input image')
ax[1].set_title('Mask image')
ax[2].set_title('RGB Mask - Expected output')
ax[0].get_xaxis().set_visible(False)
ax[0].get_yaxis().set_visible(False)
ax[1].get_xaxis().set_visible(False)
ax[1].get_yaxis().set_visible(False)
ax[2].get_xaxis().set_visible(False)
ax[2].get_yaxis().set_visible(False)
plt.show()

<font size="4">To ease the segmentation visualization, a dictionary has been incorporated to the dataset mapping each class to a specific color<font>. Below, you can see the **RGB tuple** corresponding to each class as well as the class corresponding to each number in the mask image.

In [ ]:
df = pd.read_csv('../input/semantic-drone-dataset/class_dict_seg.csv')
df

<a id="section-three"></a>
# 3. Dataset Pipeline for preprocessing

 <font size="4"> Once we know better our dataset, we will create a **data generator** using a `tf.data.Dataset` object. Which will allow us not to run out of memory while loading all the dataset at once. What we want to do is just load the original dataset in **batches** in order to pass a lower amount of data to our Neural Network setting **free RAM memory**. You can try to load all the dataset at once to realise what the problem is.<font>
 

In [ ]:
num_classes=23
H=800
W=1200

def read_image(x):
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x


def read_mask(x):
    x = cv2.imread(x, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    x = x.astype(np.int32)
    return x


def tf_dataset(x,y, batch=4):
    dataset = tf.data.Dataset.from_tensor_slices((x,y)) # Dataset object from Tensorflow
    dataset = dataset.shuffle(buffer_size=100) 
    dataset = dataset.map(preprocess) # Applying preprocessing to every batch in the Dataset object
    dataset = dataset.batch(batch) # Determine atch-size
    dataset = dataset.repeat()
    dataset = dataset.prefetch(2) # Optimization
    return dataset
        

def preprocess(x,y):
    def f(x,y):
        x = x.decode()
        y = y.decode()
        image = read_image(x)
        mask = read_mask(y)
        return image, mask
    
    image, mask = tf.numpy_function(f,[x,y],[tf.float32, tf.int32])
    mask = tf.one_hot(mask, num_classes, dtype=tf.int32)
    image.set_shape([H, W, 3])    # In the Images, number of channels = 3. 
    mask.set_shape([H, W, num_classes])    # In the Masks, number of channels = number of classes. 
    return image, mask


<font size='4'> The structure of the **Pipeline** requires the filenames and the ubication of every of them to create the **Dataset**. So, before calling the data generator we will get the **files paths**. <font>

In [ ]:
root_dir = '../input/semantic-drone-dataset/dataset/semantic_drone_dataset'
img_path = root_dir + '/original_images/'
mask_path = root_dir + '/label_images_semantic/'

names = list(map(lambda x: x.replace('.jpg', ''), os.listdir(img_path)))

<font size='4'> To make things easier we will split our data into the **training, validation and testing** set spliting the file paths array in order to waste less memory.

In [ ]:
X_trainval, X_test = train_test_split(names, test_size=0.1, random_state=19)
X_train, X_val = train_test_split(X_trainval, test_size=0.2, random_state=19)

print(f"Train Size : {len(X_train)} images")
print(f"Val Size   :  {len(X_val)} images")
print(f"Test Size  :  {len(X_test)} images")

y_train = X_train #the same values for images (X) and labels (y)
y_test = X_test
y_val = X_val

img_train = [os.path.join(img_path, f"{name}.jpg") for name in X_train]
mask_train = [os.path.join(mask_path, f"{name}.png") for name in y_train]
img_val = [os.path.join(img_path, f"{name}.jpg") for name in X_val]
mask_val = [os.path.join(mask_path, f"{name}.png") for name in y_val]
img_test = [os.path.join(img_path, f"{name}.jpg") for name in X_test]
mask_test = [os.path.join(mask_path, f"{name}.png") for name in y_test]


<font size='4'> **Testing set preprocessing is quite different!!!** So we will create another Pipeline. <font>
    


In [ ]:
## Dataset Pipeline used for testing the model
num_classes=23
H=800
W=1200


def test_dataset(x, batch=4):
    dataset = tf.data.Dataset.from_tensor_slices(x)
    dataset = dataset.map(preprocess_test)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(2)
    return dataset
        

def preprocess_test(x):
    def f(x):
        x = x.decode()
        image = read_image(x)
        return image
    
    image = tf.convert_to_tensor(tf.numpy_function(f, [x] , [tf.float32]))
    image = tf.reshape(image, (H, W, 3))    # In the Images, number of channels = 3.  
    return image

<font size='4'>To end up with preprocessing we will set the batch size (just try the maximum value that prevents your session from crashing). The intention is to put the **bigger batch size possible**: the one that will <u>optimize the training time without running out of memory</u>.<font>

In [ ]:
batch_size=3

train_dataset = tf_dataset(img_train, mask_train, batch = batch_size)
valid_dataset = tf_dataset(img_val, mask_val, batch = batch_size)

train_steps = len(img_train)//batch_size
valid_steps = len(img_val)//batch_size

<a id="section-four"></a>
# 4. Convolutional Neural Network Model and Training

<font size='4'>The model I have chosen for training is a simple U-Net. Take a look at its architecture [here](https://towardsdatascience.com/u-net-b229b32b4a71).<font>

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda


def multi_unet_model(n_classes=23, IMG_HEIGHT=800, IMG_WIDTH=1200, IMG_CHANNELS=3):
#Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = inputs

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)  # Original 0.1
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)  # Original 0.1
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
     
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.1)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
     
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.1)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
     
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    
    #Expansive path 
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.1)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
     
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
     
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)  # Original 0.1
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
     
    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)  # Original 0.1
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
     
    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(c9)
     
    model = Model(inputs=[inputs], outputs=[outputs])
    
    #NOTE: Compile the model in the main program to make it easy to test with various loss functions
    model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])
    
    model.summary()
    
    return model

<font size='4'>Let's take a glance at the `model.summary()` which will help us to analyse the **layers dimensions**. It is specially important to take care of the **Input and Output layers** as they must match our input and expected output dimensions.

In [ ]:
model = multi_unet_model()

<font size='4'>To avoid losing time training, it is always useful to set up a `EarlyStopping` callback. There are other types of callbacks which can help you to perform different tasks while training such as saving checkpoints.<font>

In [ ]:
es = tf.keras.callbacks.EarlyStopping(min_delta=0.001, patience=10)

<font size='4'>Given that this model takes a lot of time to train (6 hours more less). I have pre-trained it on my computer and I will load the pretrain model in order not to fully train it on Kaggle. Here, I will just run 1 epoch to show that it works properly.<font>

In [ ]:
model =  tf.keras.models.load_model('../input/pretrained-model/model.h5') 

In [ ]:
history = model.fit(train_dataset,
          steps_per_epoch=train_steps,
          validation_data=valid_dataset,
          validation_steps=valid_steps,
          epochs=1 #just to check it works properly
         )

<a id="section-five"></a>
# 5. Model performance
<font size='4'>To conclude, we will load our test Dataset and **evaluate our model on the test set**.<font>

In [ ]:
test_ds = tf_dataset(img_test, mask_test, batch = batch_size)
model.evaluate(test_ds, steps=14)

<font size='4'>Accuracy on the testing set is around **77%**. However, accuracy is not the better metric while doing semantic segmentation. It is commonly used the **Jaccard coefficient** also known as **MIoU (Mean Intersection over Union)**. More information about it [here](https://towardsdatascience.com/metrics-to-evaluate-your-semantic-segmentation-model-6bcb99639aa2).<font>

In [ ]:
# To visualize loss and accuracy evolution

# df = pd.DataFrame(history.history)
# df[['loss', 'acc']].plot()

<font size='4'>**Let's predict the outputs!!**<font>

In [ ]:
pred = model.predict(test_dataset(img_test, batch = 1), steps=40)

<font size='4'>Now some post-processing is required to get the **MIoU**. We will define a function to calculate the IoU knowing the values of the confusion matrix and then it is straight-forward to calculate it.<font>

In [ ]:
pred.shape

In [ ]:
predictions = np.argmax(pred, axis=3)
label = np.array([cv2.resize(cv2.imread(mask_path+img_test[i][-7:-4]+'.png')[:, :, 0], (1200, 800)) for i in range(predictions.shape[0])])
label = label.flatten()
predictions = predictions.flatten()

In [ ]:
from sklearn.metrics import confusion_matrix
color_dict = pd.read_csv('../input/semantic-drone-dataset/class_dict_seg.csv')
cm = confusion_matrix(label, predictions, labels=range(23))
df_cm = pd.DataFrame(cm,  columns=list(color_dict['name'])[:23])
df_cm

In [ ]:
def iou(cm, i):
    return cm[i,i]/(sum(cm[i])+sum(cm[:,i])-cm[i,i])

print('MIoU: {0}%'.format(round(100*np.mean(np.nan_to_num(np.array([iou(cm, i) for i in range(23)]))), 4)))

<font size='4'>**28% MIoU**... Definitely, our model can improve quite a lot but it is a good and simple approach to a **multi-class semantic segmentation problem**. <font>

# 6. Visualize the model predictions

<font size='4'>Finally, let's see some predictions. Some post-processing is required<font>

In [ ]:
cmap = np.array(list(color_dict[[' r', ' g', ' b']].transpose().to_dict('list').values()))
predictions = predictions.reshape(-1, 800, 1200)
label = label.reshape(-1, 800, 1200)

i = 18
fig, ax = plt.subplots(3, 2, figsize=(15, 15))
for j in range(3):
    ax[j, 0].imshow(cmap[predictions[i+j]])
    ax[j, 1].imshow(cmap[label[i+j]])
    ax[j, 0].set_title('Prediction')
    ax[j, 1].set_title('Ground truth')
plt.show()

### It performs quite well!!